In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging
from functools import partial
from dask.local import get_async
from dask import bag as db
from dask import array as da
import sprite2
import numpy as np
from matplotlib import pyplot as plt
import sprite2.aws
import sprite2.dask
import os

logging.debug("test")

### call a function remotely

In [11]:
def hello_func():
    return os.environ.get('AWS_LAMBDA_FUNCTION_NAME', 'dummy')

In [12]:
# invoke locally, local environment doesnt define this var
hello_func()

'dummy'

In [13]:
# invoke remotely, lamba execution enfironment does
sprite2.aws.invoke_lambda(hello_func)

'sprite'

### call a lambda remotely

In [18]:
# invoke remotely, lamba execution enfironment does
sprite2.aws.invoke_lambda(lambda: 'im executing remotely')

'im executing remotely'

### remote errors are raised locally

In [19]:
# remote exceptions raised as AwsError locally
def fun(): raise Exception('awha?')
sprite2.aws.invoke_lambda(fun)

AwsError: invalid response: {'errorMessage': 'awha?', 'errorType': 'Exception', 'stackTrace': [['/var/task/sprite2/aws.py', 62, 'executor', 'result = function()'], ['<ipython-input-19-f9b6da89c737>', 2, 'fun', '']]}

### I can even numpy remotely

In [20]:
def squareit():
    return np.square(np.arange(10))

In [21]:
squareit()

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])

In [22]:
%time sprite2.aws.invoke_lambda(squareit)

CPU times: user 3.45 ms, sys: 837 µs, total: 4.29 ms
Wall time: 63 ms


array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])

In [23]:
%timeit -n 5 sprite2.aws.invoke_lambda(squareit)

34.9 ms ± 3.55 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)
